# 1. Load and Preprocess the Data

In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set paths for training and test data
train_dir = 'data/train/'
test_dir = 'data/test/'

# Set up the image data generator for training
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical'
)

# Set up the image data generator for testing
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


# 2. Model Definition and Training

In [42]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNetV2 with pre-trained weights, excluding the top classification layers
base_model = MobileNetV2(input_shape=(48, 48, 3), include_top=False, weights='imagenet')

# Freeze the base model layers
base_model.trainable = False

# Add custom layers for emotion classification
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(7, activation='softmax')  # Adjust the number of classes to match your dataset
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 50
history = model.fit(
    train_generator,
    epochs=epochs
)


/var/folders/jf/t6w9jnc93hn7d69c7kmy6r380000gn/T/ipykernel_60372/4102682160.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(48, 48, 3), include_top=False, weights='imagenet')


Epoch 1/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 24s 25ms/step - accuracy: 0.3201 - loss: 1.7257
Epoch 2/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 22s 25ms/step - accuracy: 0.4119 - loss: 1.5135
Epoch 3/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.4641 - loss: 1.3982
Epoch 4/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 22s 25ms/step - accuracy: 0.5089 - loss: 1.3014
Epoch 5/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.5485 - loss: 1.2059
Epoch 6/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 22s 25ms/step - accuracy: 0.6096 - loss: 1.0720
Epoch 7/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.6501 - loss: 0.9679
Epoch 8/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.6906 - loss: 0.8673
Epoch 9/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.7304 - loss: 0.7647
Epoch 10/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.7615 - loss: 0.6867
Epoch 11/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - accuracy: 0.7921 - loss: 0.6061
Epoch 12/50
898/898 ━━━━━━━━━━

# 3. Evaluate the Model on the Test Dataset

In [43]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy:.2f}')


225/225 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.3942 - loss: 6.9956
Test Accuracy: 0.40


In [44]:
# Save the trained model
# model.save('ED.h5')

# Save the trained model in Keras format
model.save('ED33.keras')


In [45]:
# model = load_model('ED.h5')
from tensorflow.keras.models import load_model

# Load the trained model from the Keras format
model = load_model('ED33.keras')


In [39]:
# Step 1: Install necessary libraries
# !pip install tensorflow ipywidgets pillow

# Step 2: Import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display

# Step 3: Load your trained model
model = load_model('ED2.keras')

# Step 4: Create file upload widget
uploader = widgets.FileUpload(
    accept='.jpg,.jpeg,.png',
    multiple=False
)
display(uploader)

# Step 5: Define prediction function
def predict_image(uploaded_file):
    image_data = uploaded_file['content']
    img = Image.open(io.BytesIO(image_data)).convert('RGB')
    img = img.resize((48, 48))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array)
    emotion_index = np.argmax(predictions[0])
    
    return emotion_index

# Step 6: Set up upload callback
def on_upload_change(change):
    if uploader.value:
        uploaded_file = next(iter(uploader.value.values()))
        emotion_index = predict_image(uploaded_file)
        print(f'Predicted Emotion Index: {emotion_index}')
        emotion_labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
        print(f'Predicted Emotion: {emotion_labels[emotion_index]}')

uploader.observe(on_upload_change, names='value')


FileUpload(value=(), accept='.jpg,.jpeg,.png', description='Upload')

In [55]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image

# Load your trained model
model = load_model('ED33.keras')

def predict_image(file_path):
    # Load the image
    img = Image.open(file_path).convert('RGB')
    img = img.resize((48, 48))  # Resize to match the model's input shape
    img_array = np.array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Make prediction
    predictions = model.predict(img_array)
    emotion_index = np.argmax(predictions[0])  # Get the index of the highest probability
    
    return emotion_index

# Specify the image file path
file_path = '8.jpg'  # Replace with your image path

# Make prediction
emotion_index = predict_image(file_path)
print(f'Predicted Emotion Index: {emotion_index}')

# Map the index to emotion labels
emotion_labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
print(f'Predicted Emotion: {emotion_labels[emotion_index]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step
Predicted Emotion Index: 0
Predicted Emotion: angry
